In [ ]:
import numpy
import torch
from stable_baselines3 import SAC
import gym
import mujoco
import gym
from gym import spaces
import numpy as np
import mujoco
import mujoco.viewer
from gym.envs.registration import register


In [19]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
import gym
from stable_baselines3 import SAC

# Create the environment
env = gym.make('Pendulum-v1', render_mode='human')

# Create the SAC model
model = SAC('MlpPolicy', env, verbose=1)

# Train the model
model.learn(total_timesteps=10000)

# Save the model
model.save("sac_pendulum")

# Load the model (optional)
model = SAC.load("sac_pendulum")

# Test the model
obs, info = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs)
    obs, rewards, dones, truncated, info = env.step(action)
    if dones or truncated:
        obs, info = env.reset()
    env.render()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\thijn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


AttributeError: partially initialized module 'torch._dynamo' has no attribute 'external_utils' (most likely due to a circular import)

# Tesbot test

In [20]:
print(gym.envs.registry.keys())

dict_keys(['CartPole-v0', 'CartPole-v1', 'MountainCar-v0', 'MountainCarContinuous-v0', 'Pendulum-v1', 'Acrobot-v1', 'LunarLander-v2', 'LunarLanderContinuous-v2', 'BipedalWalker-v3', 'BipedalWalkerHardcore-v3', 'CarRacing-v2', 'Blackjack-v1', 'FrozenLake-v1', 'FrozenLake8x8-v1', 'CliffWalking-v0', 'Taxi-v3', 'Reacher-v2', 'Reacher-v4', 'Pusher-v2', 'Pusher-v4', 'InvertedPendulum-v2', 'InvertedPendulum-v4', 'InvertedDoublePendulum-v2', 'InvertedDoublePendulum-v4', 'HalfCheetah-v2', 'HalfCheetah-v3', 'HalfCheetah-v4', 'Hopper-v2', 'Hopper-v3', 'Hopper-v4', 'Swimmer-v2', 'Swimmer-v3', 'Swimmer-v4', 'Walker2d-v2', 'Walker2d-v3', 'Walker2d-v4', 'Ant-v2', 'Ant-v3', 'Ant-v4', 'Humanoid-v2', 'Humanoid-v3', 'Humanoid-v4', 'HumanoidStandup-v2', 'HumanoidStandup-v4', 'tesbot', 'tesbot_env'])


In [ ]:


class TesbotEnv(gym.Env):
    def __init__(self, render_mode='human'):
        super(TesbotEnv, self).__init__()
        
        # Load your URDF/MJCF model
        self.model = mujoco.MjModel.from_xml_path("tesbot.urdf")
        self.data = mujoco.MjData(self.model)

        # Action and observation spaces (simplified example)
        n_actuators = self.model.nu
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(n_actuators,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.model.nq + self.model.nv,), dtype=np.float32)

        # Viewer setup (optional)
        if render_mode == 'human':
            self.viewer = mujoco.viewer.launch_passive(self.model, self.data)

    def reset(self):
        mujoco.mj_resetData(self.model, self.data)
        return self._get_obs()

    def step(self, action):
        mujoco.mj_set_control(self.model, self.data, action)
        mujoco.mj_step(self.model, self.data)
        obs = self._get_obs()
        reward = self._compute_reward()
        done = False  # Customize based on task
        return obs, reward, done, {}

    def _get_obs(self):
        return np.concatenate([self.data.qpos, self.data.qvel])

    def _compute_reward(self):
        return 1.0  # Placeholder

    def render(self, mode='human'):
        if hasattr(self, 'viewer'):
            self.viewer.sync()

    def close(self):
        if hasattr(self, 'viewer'):
            self.viewer.close()



In [22]:
# Register the environment with a unique ID
register(
    id='tesbot_env',
    entry_point='__main__:TesbotEnv',
    max_episode_steps=1000,
)
# Create the environment
env = gym.make('tesbot_env')
model = SAC('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=100_000)

# Save the model
model.save("tesbot_sac")


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\thijn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


AttributeError: partially initialized module 'torch._dynamo' has no attribute 'external_utils' (most likely due to a circular import)